In [1]:
import numpy as np
import pandas as pd

# 让画图内置
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# 忽略版本号之类的warnings
import warnings
warnings.filterwarnings('ignore')
import os

# 数据导入

In [2]:
# 导入数据
original_data = pd.read_excel('JD_data\京东消费者分析数据.xlsx')

In [3]:
# 数据量
data = original_data.copy()
data.shape

(183828, 20)

In [4]:
data.head()

,customer_id,product_id,action_date,action_id,type,age_range,gender,customer_register_date,customer_level,city_level,brand,shop_id,category,product_market_date,vender_id,fans_number,vip_number,shop_register_date,shop_category,shop_score
0,976695,328360,2018-03-05 08:52:00,10633744,PageView,5.0,W,2017-06-17,6,3.0,LukFook,2959,Bracelet,2017-10-05,4322,1159,36596,2017-07-04,Jewellery Accessories,9.043610
1,395399,141632,2018-03-20 10:55:00,5472028,PageView,4.0,M,2018-01-21,6,5.0,illuma,4203,Milk Power,2015-06-12,3563,2411,65893,2013-08-08,Mother and Infant,9.537151
2,1068082,5924,2018-04-01 13:56:00,3853455,PageView,6.0,W,2012-08-13,1,4.0,Li-Ning,8002,Badminton,2013-06-14,1457,213610,357580,2013-05-23,Outdoor Sports,9.571139
3,1068082,10723,2018-04-01 13:45:00,3853455,PageView,6.0,W,2012-08-13,1,4.0,Li-Ning,8002,Badminton,2011-07-19,1457,213610,357580,2013-05-23,Outdoor Sports,9.571139
4,735392,4559,2018-02-06 20:25:00,9434940,PageView,6.0,M,2009-09-15,7,5.0,Other,6944,Cookie,2015-11-16,355,63969,85597,2014-10-31,Food,9.665680


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183828 entries, 0 to 183827
Data columns (total 20 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   customer_id             183828 non-null  int64         
 1   product_id              183828 non-null  int64         
 2   action_date             183828 non-null  datetime64[ns]
 3   action_id               183828 non-null  int64         
 4   type                    183828 non-null  object        
 5   age_range               183745 non-null  float64       
 6   gender                  183828 non-null  object        
 7   customer_register_date  183828 non-null  datetime64[ns]
 8   customer_level          183828 non-null  int64         
 9   city_level              183703 non-null  float64       
 10  brand                   183828 non-null  object        
 11  shop_id                 183828 non-null  int64         
 12  category                183828

In [6]:
# 有意义的数值型变量的信息
data.describe()[['age_range','customer_level','city_level','fans_number','vip_number','shop_score','action_date','customer_register_date','product_market_date','shop_register_date']] 

,age_range,customer_level,city_level,fans_number,vip_number,shop_score,action_date,customer_register_date,product_market_date,shop_register_date
count,183745.000000,183828.000000,183703.000000,1.838280e+05,1.838280e+05,183828.000000,183828,183828,183828,112474
mean,4.680073,4.965734,3.292701,6.925701e+04,1.012062e+05,5.716299,2018-03-08 03:19:10.532345344,2014-07-03 05:31:32.943403520,2016-04-18 23:16:38.524707840,2015-11-13 04:00:30.983160576
min,1.000000,1.000000,1.000000,0.000000e+00,0.000000e+00,-1.000000,2018-02-01 00:00:00,2003-06-13 00:00:00,2008-05-16 00:00:00,2011-03-14 00:00:00
25%,5.000000,5.000000,3.000000,0.000000e+00,0.000000e+00,0.000000,2018-02-15 23:00:00,2012-07-30 00:00:00,2015-07-30 00:00:00,2014-11-21 00:00:00
50%,5.000000,6.000000,4.000000,4.612000e+03,1.330000e+04,9.311070,2018-03-07 21:22:30,2014-10-27 00:00:00,2016-10-25 00:00:00,2016-01-05 00:00:00
75%,6.000000,7.000000,4.000000,4.319000e+04,9.424900e+04,9.539181,2018-03-25 19:28:00,2016-07-21 00:00:00,2017-08-06 00:00:00,2017-03-09 00:00:00
max,6.000000,7.000000,6.000000,9.293487e+06,1.384168e+07,10.000000,2018-04-15 23:59:00,2018-04-15 00:00:00,2018-04-14 00:00:00,2018-04-12 00:00:00
std,1.526474,2.334717,1.411516,2.448230e+05,2.648308e+05,4.653295,NaN,NaN,NaN,NaN


- 问题：fans_number最大为 9293487， vip_number最大值为 13841676 是同一个shop 10393 

# 预处理

## 空值 

- shop_register_date 缺失值太多,在研究店铺相关问题时尽量不用

In [7]:
data.isnull().sum()

customer_id                   0
product_id                    0
action_date                   0
action_id                     0
type                          0
age_range                    83
gender                        0
customer_register_date        0
customer_level                0
city_level                  125
brand                         0
shop_id                       0
category                      0
product_market_date           0
vender_id                     0
fans_number                   0
vip_number                    0
shop_register_date        71354
shop_category                 0
shop_score                    0
dtype: int64

**age_range 空值处理**
- 可以看到 age_range 为 5 的最多，空值就用 5 填充。

In [8]:
data['age_range'].value_counts() # 5 最多

age_range
5.0    80064
6.0    59992
2.0    16077
1.0    14309
4.0    13296
3.0        7
Name: count, dtype: int64

In [9]:
data['age_range'].fillna(5, inplace=True)
data['age_range'].isnull().sum()   # 0

0

**city_level的空值处理**


In [10]:
data['city_level'].value_counts() # 4 最多
data['age_range'].fillna(4, inplace=True)


## 重复值

**action_id字段**
-  照理来说，action_id 应该是主键，表示每个人的行为编号唯一值

- 但是 action_id 却有很多重复值，这些重复的 action_id 对应的record却不是完全重复的，需要寻找新的主键

In [11]:
# action_id 也有重复值，不是主键
data['action_id'].value_counts()[0:10]

action_id
10439703    9176
5335532        7
9401782        7
9656978        6
355392         6
8220611        6
3022398        6
12053          6
898733         6
2680391        5
Name: count, dtype: int64

In [45]:
# action_id 重复但是记录却不完全重复
data.query('action_id == 10439703').iloc[:5,:]

,customer_id,product_id,action_date,action_id,type,age_range,gender,customer_register_date,customer_level,city_level,...,shop_id,category,product_market_date,vender_id,fans_number,vip_number,shop_register_date,shop_category,shop_score,action_day
89941,796801,171645,2018-02-07 18:45:00,10439703,Comment,2.0,M,2015-08-20,5,5.0,...,9445,Phone,2013-05-11,3666,0,0,NaT,Electronics,0.0,2018-02-07
89942,750076,230884,2018-03-19 14:19:00,10439703,Comment,5.0,M,2014-08-15,5,4.0,...,3388,Phone,2012-03-26,3666,0,0,NaT,Electronics,0.0,2018-03-19
89943,279373,171645,2018-03-22 16:04:00,10439703,Comment,6.0,M,2010-03-25,7,1.0,...,9445,Phone,2013-05-11,3666,0,0,NaT,Electronics,0.0,2018-03-22
89944,322424,171645,2018-03-19 08:51:00,10439703,Comment,5.0,M,2011-10-15,5,4.0,...,9445,Phone,2013-05-11,3666,0,0,NaT,Electronics,0.0,2018-03-19
89945,101163,220537,2018-02-11 00:38:00,10439703,Comment,5.0,W,2015-06-12,5,5.0,...,157,Phone,2013-04-26,3666,0,0,NaT,Electronics,0.0,2018-02-11


**寻找符合主键**
- groupby(['customer_id', 'product_id','type','action_date']) 后的数据仅有 2 个重复值, 且这两个重复值除了action_id其他都一样
- 可以认为这4个字段组成一个符合主键 ，且这俩记录重复的，删掉即可
- 含义就是：用户 在某个时间 对商品 ，做了什么

In [13]:
# 用groupby找复合主键
grouped = data.groupby(['customer_id', 'product_id','type','action_date']).size()
grouped[grouped > 1]

customer_id  product_id  type      action_date        
767393       86843       PageView  2018-03-26 19:01:00    2
dtype: int64

In [14]:
data.query("customer_id == 767393 and product_id == 86843")
# 这两条记录除了action_id，其他都一样，说明：1.action_id是个傻逼， 2.这两条记录是重复值，应当要删掉

,customer_id,product_id,action_date,action_id,type,age_range,gender,customer_register_date,customer_level,city_level,brand,shop_id,category,product_market_date,vender_id,fans_number,vip_number,shop_register_date,shop_category,shop_score
33413,767393,86843,2018-03-26 19:01:00,6861957,PageView,6.0,W,2014-02-12,1,4.0,Microsoft,4296,Xbox,2016-06-14,3666,0,0,NaT,Electronics,0.0
112372,767393,86843,2018-03-26 19:01:00,8915506,PageView,6.0,W,2014-02-12,1,4.0,Microsoft,4296,Xbox,2016-06-14,3666,0,0,NaT,Electronics,0.0


In [15]:
# 删除重复值
to_be_drop = data.query("customer_id == 767393 and product_id == 86843").iloc[0]
data.drop(to_be_drop.name, inplace=True)


In [16]:
# 确认删除情况
data.query("customer_id == 767393 and product_id == 86843")
grouped = data.groupby(['customer_id', 'product_id','type','action_date']).size()
grouped[grouped > 1]

,customer_id,product_id,action_date,action_id,type,age_range,gender,customer_register_date,customer_level,city_level,brand,shop_id,category,product_market_date,vender_id,fans_number,vip_number,shop_register_date,shop_category,shop_score
112372,767393,86843,2018-03-26 19:01:00,8915506,PageView,6.0,W,2014-02-12,1,4.0,Microsoft,4296,Xbox,2016-06-14,3666,0,0,NaT,Electronics,0.0


# 异常值探查

**字段探查**
- 内容：字段有哪些取值？取值范围？取值的分布？

- 分类变量：
    - 取值多的看取值数量（distinct user_id），
    - 取值少的具体的取值（gender 的 value_counts）

- 时间变量：
    - 看时间范围，按年/月/日的分布，唯一日期/年份数量

- 连续变量：看分布，是否存在异常值，是否存在空值

- 看数据分布时，用 value_counts；拆解维度的时候再考虑groupby(column)


## 用户行为

**customer_id**  
- 去重后用户数: 140715


In [40]:
# 用户数，去重后
print("去重后用户数:",data['customer_id'].nunique())

去重后用户数: 140715


**product_id**
- 去重后商品数: 52950

In [41]:
# 商品数，去重后
print("去重后商品数:",data['product_id'].nunique())


去重后商品数: 52950


**action_date**  
- datetime类型,从2018-2-1到2018-4-15

In [42]:
# action_date探索
print("action_date的格式：")
print(data['action_date'].head(2),end='\n\n')

# 时间范围
print("最早日期：",data['action_date'].min().date())
print("最晚日期：",data['action_date'].max().date(),end='\n\n')

# 日期分布
data['action_date'].dt.date.sort_values().unique().shape[0]  # 74天 每天都有数据

action_date的格式：
0   2018-03-05 08:52:00
1   2018-03-20 10:55:00
Name: action_date, dtype: datetime64[ns]

最早日期： 2018-02-01
最晚日期： 2018-04-15



**action_id**
- 傻逼东西，没用

**type** 
- 行为类别，有PageView,Follow,SaveCart,Order,Comment这5种取值

- Order明显比saveCart多，这不符合常理
    - 发现 SaveCart 在 2018-4-8之后才有数据，可能是数据错误，也可能是4-8之后才有购物车功能。
    -  但是 4-8之后每天SaveCart都比除了PV之外的数据高，不太可能是新增的

- Follow数较小，应该指的是收藏店铺 ； Comment指的是评论商品，在购买商品之后 ；用户路径时只需要都不需要考虑 

- 需要过滤掉 4-8 之前的数据


In [21]:
# 行为类型的分布情况
print("行为类型的分布情况")
print(data['type'].value_counts())

行为类型的分布情况
type
PageView     163953
Order         10698
Comment        4010
SavedCart      2987
Follow         2179
Name: count, dtype: int64


In [22]:
# 探究为什么SavedCart的次数比Order的次数少

# 新增一列 action_day，表示行为发生的日期
data['action_day']  = data['action_date'].apply(lambda x: x.date())

# 统计每天每个行为的次数
type_count = data.groupby(['action_day', 'type']).size().unstack()
type_count

# 找到有购物车的记录去日期
type_count[type_count['SavedCart'].isnull().apply(lambda x: not x)]


type,Comment,Follow,Order,PageView,SavedCart
action_day,,,,,
2018-04-08,70.0,37.0,156.0,1899.0,320.0
2018-04-09,70.0,32.0,169.0,2069.0,425.0
2018-04-10,71.0,26.0,156.0,2111.0,421.0
2018-04-11,75.0,30.0,162.0,2049.0,434.0
2018-04-12,58.0,31.0,146.0,1850.0,363.0
2018-04-13,65.0,34.0,146.0,1920.0,354.0
2018-04-14,56.0,35.0,159.0,1892.0,356.0
2018-04-15,50.0,33.0,135.0,1695.0,314.0


In [23]:
# 4-8之后的数据只有20474条
from datetime import date

recent_data = data[data['action_day']>=date(2018, 4, 8)]
recent_data.shape


(20474, 21)

## 用户信息

**age_range**
- 有 6 个年龄段，数字代表 1-7

In [46]:
# 年龄的分布情况
print("年龄的分布情况")
print(data.value_counts('age_range'))

年龄的分布情况
age_range
5.0    80147
6.0    59991
2.0    16077
1.0    14309
4.0    13296
3.0        7
Name: count, dtype: int64


**gender**
- 性别：男M:120077，女W:63277，未知U:473

In [55]:
# 性别的分布情况
print("性别的分布情况")
print(data.value_counts('gender'))

性别的分布情况
gender
M    120077
W     63277
U       473
Name: count, dtype: int64


**customer_register_date** 
- 用户注册的时间范围：2003-06-13 ~ 2018-04-15 
- 注册年份分布合理，非重复日期数 4104 （10年多，合理）

In [59]:
# 时间范围
print("最早日期：",data['customer_register_date'].min().date())
print("最晚日期：",data['customer_register_date'].max().date(),end='\n\n')

# 注册时间按年分布
print("注册时间按年分布")
print(data.groupby(data['customer_register_date'].dt.year)['customer_id'].size(),end='\n\n')

# 去重后日期数
print("去重后日期数：",data['customer_register_date'].nunique())


最早日期： 2003-06-13
最晚日期： 2018-04-15

注册时间按年分布
customer_register_date
2003       11
2004       40
2005       77
2006      200
2007      485
2008     1882
2009     4391
2010     9821
2011    16845
2012    21406
2013    19854
2014    21473
2015    26422
2016    26500
2017    25834
2018     8586
Name: customer_id, dtype: int64

去重后日期数： 4104


**customer_level**
- 会员级别
- 可以研究会员级别和行为之间的关系，不同等级会员都在干嘛

In [27]:
# 会员级别的分布情况
print("会员级别的分布情况")
print(data.groupby('customer_level')['customer_id'].nunique(),end='\n\n')

会员级别的分布情况
customer_level
1    35864
2        2
3       65
4      108
5    35160
6    21379
7    48137
Name: customer_id, dtype: int64



**city_level**
- 会员城市级别
- 有125个空值，无伤大雅，用众数填了就行
- 可以研究用户的城市分布

In [48]:
# 城市等级的分布情况
print("城市等级的分布情况")
print(data.groupby('city_level')['customer_id'].nunique(),end='\n\n')


城市等级的分布情况
city_level
1.0    29990
2.0     1816
3.0    36284
4.0    41931
5.0    30150
6.0      433
Name: customer_id, dtype: int64

空值: 125


## 商品

**brand**
- 产品品牌
- 72个品牌

In [29]:
# 产品品牌数
print("产品品牌数:",data['brand'].nunique(),end='\n\n')

# 空值
print("空值:",data['brand'].isnull().sum())


产品品牌数: 72

空值: 0


**category**
- 产品类别
- 53 个类别

In [30]:
# 产品种类数
print("产品种类数:",data['category'].nunique(),end='\n\n')

# 空值
print("空值:",data['category'].isnull().sum())

产品种类数: 53

空值: 0


In [31]:
# 每个品类，每个品牌有多少商品

product_count = data.groupby(['category','brand'])['product_id'].nunique()
product_count
# product_count.to_csv('product_count.csv')

category          brand     
Air Conditioner   Gree          608
                  Midea         361
Badminton         Li-Ning       414
Basketball        Li-Ning       275
Basketball Shoes  Anta           25
                               ... 
Washing Machine   Haier          31
                  Konka         212
                  LittleSwan    543
                  Midea         203
Xbox              Microsoft     429
Name: product_id, Length: 227, dtype: int64

**product_market_date**
- 产品上市日期
- 可以研究 "用户是不是喜欢买新上市的产品"

In [32]:
# custom_register_date
print("product_market_date的格式")
print(data['product_market_date'].head(2),end='\n\n')

# 时间范围
print("最早日期：",data['product_market_date'].min().date())
print("最晚日期：",data['product_market_date'].max().date(),end='\n\n')

# 商品上架时间按年分布
print("商品上架时间按年分布")
print(data.groupby(data['product_market_date'].dt.year)['product_id'].nunique())

product_market_date的格式
0   2017-10-05
1   2015-06-12
Name: product_market_date, dtype: datetime64[ns]

最早日期： 2008-05-16
最晚日期： 2018-04-14

商品上架时间按年分布
product_market_date
2008        3
2009       40
2010      211
2011      285
2012      709
2013     1644
2014     3050
2015     6182
2016    11988
2017    23757
2018     5081
Name: product_id, dtype: int64


## 店铺信息

**shop_id & vender_id** 
- 店铺编码 和 商家编码 
- shop_id 和 vender_id 一一对应，共有6086个店铺

In [33]:
# 证明 shop_id 和 vender_id 一一对应
combin = data.groupby(['shop_id']).agg({'vender_id':'nunique'}) # 计算每个shop_id对应的vender_id的数量
combin['vender_id']>1                                    # 判断每个 shop_id 有没有对应大于1个 vender_id
print(sum(combin['vender_id']>1))                        # 0 表示所有 shop_id 都有对应1个 vender_id

0


In [34]:
# 去重店铺数
print("去重店铺数:",data['shop_id'].nunique())
# 空值
print("空值:",data['shop_id'].isnull().sum())

去重店铺数: 6086
空值: 0


**fans_number & vip_number**
- 粉丝数 和 会员数
- 原生的统计特征，可以用来研究和其他指标的相关性

In [35]:
print("粉丝数为0的记录数：",(data['fans_number']==0).sum())
print("vip数为0的记录数：",(data['vip_number']==0).sum())

粉丝数为0的记录数： 71354
vip数为0的记录数： 71356


**shop_register_date** 
- 开店时间
- 71354个空值

In [36]:
print("空值：",data['shop_register_date'].isnull().sum())

空值： 71353


**shop_category** 
- 店铺主营

In [37]:
data['shop_category'].unique()

array(['Jewellery Accessories', 'Mother and Infant', 'Outdoor Sports',
       'Food', 'Electronics', 'Beauty Makeup', 'Clothes', 'Furniture',
       'Household Eletric Appliance'], dtype=object)

**shop_score**
- 店铺打分

In [38]:
print("店铺评分为0的记录数：",(data['shop_score']==0).sum())

店铺评分为0的记录数： 71354


# 异常值统一处理